In [1]:
 import tempfile
 import timeit
 from copy import deepcopy

 import astropy.units as u
 import matplotlib.pyplot as plt
 import numpy as np
 from astropy.coordinates import AltAz, angular_separation
 from matplotlib.colors import ListedColormap
 from scipy.sparse.csgraph import connected_components
 from traitlets.config import Config

 from ctapipe.calib import CameraCalibrator
 from ctapipe.image import (
     ImageProcessor,
     camera_to_shower_coordinates,
     concentration_parameters,
     hillas_parameters,
     leakage_parameters,
     number_of_islands,
     timing_parameters,
     toymodel,
 )
 from ctapipe.image.cleaning import tailcuts_clean
 from ctapipe.io import DataWriter, EventSource, TableLoader
 from ctapipe.reco import ShowerProcessor
 from ctapipe.utils.datasets import get_dataset_path
 from ctapipe.visualization import ArrayDisplay, CameraDisplay

/data/home/zzp/miniforge3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
source = EventSource("simtel.zst", max_events=5)
calibrator = CameraCalibrator(subarray=source.subarray)
image_processor = ImageProcessor(subarray=source.subarray)
shower_processor = ShowerProcessor(subarray=source.subarray)
horizon_frame = AltAz()

for event in source:
    calibrator(event)
    image_processor(event)
    shower_processor(event)

    true_alt = event.simulation.shower.alt
    true_az = event.simulation.shower.az
    true_core_x = event.simulation.shower.core_x
    true_core_y = event.simulation.shower.core_y

    stereo = event.dl2.stereo.geometry["HillasReconstructor"]
    if stereo.is_valid:
        reco_alt = stereo.alt
        reco_az = stereo.az
        reco_core_x = stereo.core_x
        reco_core_y = stereo.core_y

        print(f"True Direction: Alt = {true_alt:.6f}, Az = {true_az:.6f}")
        print(f"Reco Direction: Alt = {reco_alt:.6f}, Az = {reco_az:.6f}")
        print(f"True Core: X = {true_core_x:.6f}, Y = {true_core_y:.6f}")
        print(f"Reco Core: X = {reco_core_x:.6f}, Y = {reco_core_y:.6f}")
        print("---")


True Direction: Alt = 1.570796 rad, Az = 3.141593 rad
Reco Direction: Alt = 1.570553 rad, Az = 5.067020 rad
True Core: X = -13.301501 m, Y = 92.201797 m
Reco Core: X = -13.358181 m, Y = 91.410790 m
---
True Direction: Alt = 1.570796 rad, Az = 3.141593 rad
Reco Direction: Alt = 1.570651 rad, Az = 2.283632 rad
True Core: X = 64.319771 m, Y = 147.590378 m
Reco Core: X = 65.134346 m, Y = 149.353273 m
---
True Direction: Alt = 1.570796 rad, Az = 3.141593 rad
Reco Direction: Alt = 1.570715 rad, Az = 2.304140 rad
True Core: X = -124.684914 m, Y = 111.200752 m
Reco Core: X = -124.746490 m, Y = 111.584049 m
---
True Direction: Alt = 1.570796 rad, Az = 3.141593 rad
Reco Direction: Alt = 1.570474 rad, Az = 1.262830 rad
True Core: X = 88.302597 m, Y = -100.900871 m
Reco Core: X = 88.131136 m, Y = -99.914515 m
---
True Direction: Alt = 1.570796 rad, Az = 3.141593 rad
Reco Direction: Alt = 1.570744 rad, Az = 2.523028 rad
True Core: X = 140.158081 m, Y = -96.490456 m
Reco Core: X = 140.753185 m, Y = 